In [2]:
import tensorflow as tf
from kerashypetune import KerasGridSearch
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

In [42]:
stfold = StratifiedKFold(n_splits=5, shuffle=True)
train_l = list()
val_l = list()
test_l = list()
maxlen = 100
cnt = 0
for train_index, test_index in stfold.split(data['comment'], data['label']):
    cnt += 1
    train = data.iloc[train_index]
    train_x, train_y = train['comment'], train['label']
    tmp = data.iloc[test_index]

    val, test = train_test_split(tmp, test_size=0.5, stratify=tmp['label'])    
    val_x, val_y = val['comment'], val['label']
    test_x, test_y = test['comment'], test['label']    
    
    train_x = keras.preprocessing.sequence.pad_sequences(train_x, maxlen=maxlen)
    val_x = keras.preprocessing.sequence.pad_sequences(val_x, maxlen=maxlen)
    test_x = keras.preprocessing.sequence.pad_sequences(test_x, maxlen=maxlen)
    
    embed_dim = 128  # Embedding size for each token
    vocab_size = n_word_unique + 1

    inputs = layers.Input(shape=(maxlen, ))
    embedding_layer = layers.Embedding(input_dim = vocab_size, output_dim = embed_dim, input_length=maxlen)
    x = embedding_layer(inputs)
    x = layers.Bidirectional(layers.GRU(64, return_sequences = True))(x)
    x = layers.Conv1D(64, kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(x)
    x = layers.Dropout(0.3)(x)
    avg_pool = layers.GlobalAveragePooling1D()(x)
    max_pool = layers.GlobalMaxPooling1D()(x)
    x = layers.concatenate([avg_pool, max_pool])
    outputs = layers.Dense(1, activation="sigmoid")(x)

    model = keras.Model(inputs=inputs, outputs=outputs)
    model_path = "sigmoid_models/{itr:04}/".format(itr=cnt)
    checkpoint_path = model_path + "{epoch:04d}.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)
    !mkdir sigmoid_models
    !mkdir $model_path
    
    
    model.compile('adam', "binary_crossentropy", metrics=["accuracy", 
                                                          tf.keras.metrics.AUC(name='auc')])
    
    

    model.save_weights(checkpoint_path.format(epoch=0))

    checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_auc', verbose=1,
            save_weights_only=True, save_best_only=True, mode='max')
    
    history = model.fit(
                        train_x, train_y, 
                        batch_size=128, epochs=10, 
                        validation_data=(val_x, val_y),
                        callbacks = [checkpoint]
                        )

    latest = tf.train.latest_checkpoint(checkpoint_dir)
    model.load_weights(latest)
    
    train_eval = model.evaluate(train_x, train_y)
    val_eval = model.evaluate(val_x, val_y)
    test_eval = model.evaluate(test_x, test_y)
    
    test_l.append(test_eval)
    val_l.append(val_eval)
    train_l.append(train_eval)

Epoch 1/10
63/63 [==============================] - ETA: 0s - loss: 0.6269 - accuracy: 0.6449 - auc: 0.7114
Epoch 00001: val_auc improved from -inf to 0.88892, saving model to sigmoid_models/0001/0001.ckpt
63/63 [==============================] - 12s 190ms/step - loss: 0.6269 - accuracy: 0.6449 - auc: 0.7114 - val_loss: 0.4395 - val_accuracy: 0.8080 - val_auc: 0.8889
Epoch 2/10
63/63 [==============================] - ETA: 0s - loss: 0.1784 - accuracy: 0.9374 - auc: 0.9802
Epoch 00002: val_auc improved from 0.88892 to 0.92197, saving model to sigmoid_models/0001/0002.ckpt
63/63 [==============================] - 11s 174ms/step - loss: 0.1784 - accuracy: 0.9374 - auc: 0.9802 - val_loss: 0.3403 - val_accuracy: 0.8760 - val_auc: 0.9220
Epoch 3/10
63/63 [==============================] - ETA: 0s - loss: 0.0231 - accuracy: 0.9944 - auc: 0.9992
Epoch 00003: val_auc did not improve from 0.92197
63/63 [==============================] - 11s 177ms/step - loss: 0.0231 - accuracy: 0.9944 - auc: 0.

In [43]:
test_l = np.array(test_l)
val_l = np.array(val_l)
train_l = np.array(train_l)

print("test avg loss: ", np.mean(test_l[:, 0]), "+/-" ,np.std(test_l[:, 0]))
print("test avg acc: ", np.mean(test_l[:, 1]),  "+/-" ,np.std(test_l[:, 1]))
print("test avg auc: ", np.mean(test_l[:, 2]),  "+/-" ,np.std(test_l[:, 2]))

print('\n')


print("val avg loss: ", np.mean(val_l[:, 0]), "+/-" ,np.std(val_l[:, 0]))
print("val avg acc: ", np.mean(val_l[:, 1]),  "+/-" ,np.std(val_l[:, 1]))
print("val avg auc: ", np.mean(val_l[:, 2]),  "+/-" ,np.std(val_l[:, 2]))


print('\n')


print("train avg loss: ", np.mean(train_l[:, 0]), "+/-" ,np.std(train_l[:, 0]))
print("train avg acc: ", np.mean(train_l[:, 1]),  "+/-" ,np.std(train_l[:, 1]))
print("train avg auc: ", np.mean(train_l[:, 2]),  "+/-" ,np.std(train_l[:, 2]))

test avg loss:  0.3271672070026398 +/- 0.054687655721903296
test avg acc:  0.8848000049591065 +/- 0.010225464574808362
test avg auc:  0.9374139666557312 +/- 0.007848048294543108


val avg loss:  0.33187201619148254 +/- 0.024418608842521564
val avg acc:  0.8863999962806701 +/- 0.008708607241584546
val avg auc:  0.9339932560920715 +/- 0.008471720819080493


train avg loss:  0.04705483571160585 +/- 0.023019229988576452
train avg acc:  0.9907250046730042 +/- 0.005290564489527929
train avg auc:  0.9991085886955261 +/- 0.0005412977687073082
